In [82]:
import pandas as pd
import rioxarray
import xarray as xr
from netCDF4 import Dataset
import geopandas
from utilities.weatherFunctions import *

# Measuring change in maximum and minimum temperature

In [83]:
ds_tasmax = xr.open_mfdataset("../../data/gldas/tasmax/wld_cli_gldas2*", parallel = True)
ds_tasmin = xr.open_mfdataset('../../data/gldas/tasmin/wld_cli_gldas2*', parallel=True)
#ds_tasmin = xr.open_mfdataset('../../data/gldas/')

In [ ]:
def clip_area(ds, area):

    ds.rio.write_crs("EPSG:4326", inplace=True)
    clipped_area = ds.rio.clip(area.geometry.values,all_touched=True, drop=True)

    return clipped_area

#### Choose Area of Interest

In [84]:
INDIA = geopandas.read_file("../../data/shapefiles/india_district/sh819zz8121.shp").to_crs("EPSG:4326")
CHENNAI = INDIA[INDIA["laa"] == "CHENNAI"]

In [88]:
ds_tasmax=ds_tasmax.drop(['time_bnds'])
ds_tasmin=ds_tasmin.drop(['time_bnds'])
chennai_tmax = clip_area(ds_tasmax, CHENNAI)
chennai_tmin = clip_area(ds_tasmin, CHENNAI)


In [90]:
chennai_tmax['year'] = chennai_tmax['time'].dt.year
chennai_tmax['Tasmax_C'] = chennai_tmax['Tasmax'] - 273

chennai_tmin['year'] = chennai_tmin['time'].dt.year
chennai_tmin['Tasmin_C'] = chennai_tmin['Tasmin'] - 273

In [93]:
chennai_tmax

<xarray.Dataset>
Dimensions:      (lat: 2, lon: 2, time: 27149)
Coordinates:
  * lat          (lat) float64 12.88 13.12
  * lon          (lon) float64 80.12 80.38
  * time         (time) datetime64[ns] 1948-01-01T13:30:00 ... 2022-04-30T13:...
    spatial_ref  int64 0
Data variables:
    Tasmax       (time, lat, lon) float32 dask.array<chunksize=(366, 2, 2), meta=np.ndarray>
    year         (time) int64 1948 1948 1948 1948 1948 ... 2022 2022 2022 2022
    Tasmax_C     (time, lat, lon) float32 dask.array<chunksize=(366, 2, 2), meta=np.ndarray>
Attributes: (12/21)
    CDI:                    Climate Data Interface version 1.9.10 (https://mp...
    Conventions:            CF-1.6
    source:                 Noah_v3.6 forced with Princeton_V2.2
    institution:            NASA GSFC
    missing_value:          -9999.0
    tavg definision::       past 3-hour average
    ...                     ...
    DX:                     0.25
    DY:                     0.25
    history_L34RS:          'Created by L34RS v1.4.2 @ NASA GES DISC on Septe...
    frequency:              day
    history:                Sun Nov 13 12:06:30 2022: cdo -z zip_5 -setattrib...
    CDO:                    Climate Data Operators version 1.9.10 (https://mp...

In [95]:
import hvplot.dask
import hvplot.xarray

chennai_tmax.hvplot.line(x='time', y='Tasmax')